# Efficient Algorithms for Calculating Net Present Value

## PV for the cash flow C1 , C2 , C3 ,......Cn and y is the yield.

\begin{equation*}
\text{PV} = \frac{C_1}{(1+y)}+\frac{C_2}{(1+y)^2}+\cdots+\frac{C_n}{(1+y)^n} 
\end{equation*}

In [1]:
C = collect(1:1:100000) #Creating a random sequence of Cash FLows using collect function in Julia
length(C)

100000

In [2]:
PV = 0
PVnew = 0
y = 0.01

0.01

# Below is the basic raw implementation of Eq.1 
\begin{equation*}
\text{Time complexity} = O(n^2)
\end{equation*}

In [3]:
@time( 
    for i in 1:length(C)
        PV = PV + C[i]/(1+y)^i
    end
        )

  0.057512 seconds (799.05 k allocations: 13.744 MiB, 14.22% gc time)


# Using Horner's Method of Polynomial Evaluation 
\begin{equation*}
\text{Time complexity} = O(n)
\end{equation*}

\begin{equation*}
\text{PV} = [[(\frac{C_n}{1+y}+Cn-1)\frac{1}{1+y}+Cn-2]\frac{1}{1+y}+\cdots]\frac{1}{1+y}
\end{equation*}


In [4]:
@time(
   for j in length(C):-1:1
        PVnew = (PVnew + C[j])/(1+y)
   end
)

  0.025587 seconds (698.47 k allocations: 12.184 MiB)


In [5]:
PV

10099.999999999916

In [6]:
PVnew

10099.9999999999

# Numerical Methods for Finding Yields


## Solve the Equation for y  :-
\begin{equation*}
f(y) = \sum\frac{C_t}{(1+y)^t} - P = 0
\end{equation*}

## Newton Ralphson Method
\begin{equation*}
x_{n+1} = x_n - \frac{f(x)}{f'(x)}
\end{equation*}
 
\begin{equation*}
\text{First order derivative:-}
f'(x) = -\sum\frac{tC}{(1+x)^{t+1}}
\end{equation*}
 

In [7]:
function NPV(y,C,P,n)
    NPV = 0
    for i in n:-1:1
        NPV = (NPV+C)/(1+y)
    end
    P = NPV - P
    return P
end

NPV (generic function with 1 method)

In [8]:
f = NPV(0.01,10,100,10)

-5.286954692983301

In [9]:
function NPVprime(y,C,P,n)
    NPVprime = 0
    for i in n:-1:1
        NPVprime = NPVprime+ ((i*C)/(1+y)^(1+i))
    end
    return -NPVprime
end

NPVprime (generic function with 1 method)

In [10]:
fprime = NPVprime(0.01,10,100,10)

-508.06735552361886

## Custom Function for Newton Ralphson Method

In [11]:
function newton(f::Function, x0::Number, fprime::Function, args::Tuple=();
                tol::AbstractFloat=1e-8, maxiter::Integer=50, eps::AbstractFloat=1e-10)    
    for _ in 1:maxiter
        yprime = fprime(x0, args...)
        if abs(yprime) < eps
            warn("First derivative is zero")
            return x0
        end
        y = f(x0, args...)
        x1 = x0 - y/yprime
        if abs(x1-x0) < tol
            return x1
        end
        x0 = x1
    end
    error("Max iteration exceeded")
end

newton (generic function with 2 methods)

\begin{equation*}
\text{Find roots for the equation:-}
f(x) = x^2-2
\end{equation*}

In [12]:
(f1(x)=x^2-2; f1prime(x)=2x; x0=1; newton(f1,x0,f1prime))

1.4142135623730951

## Yield y  for C = 5, P = 100 , n = 10

In [13]:
y = newton(NPV,0,NPVprime,(5,100,10))

-0.10956029368474335

In [14]:
NPV(y,5,100,10)

2.842170943040401e-14

# Summary 
## In this notebook we have learned about finding NPV and also yield from given NPV.  